In [1]:
!pip install opencv-python numpy



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np

# Define the quadrants
def get_quadrant(x, y, frame_width, frame_height):
    if x < frame_width / 2 and y < frame_height / 2:
        return 1
    elif x >= frame_width / 2 and y < frame_height / 2:
        return 2
    elif x < frame_width / 2 and y >= frame_height / 2:
        return 3
    else:
        return 4

def track_balls(video_path, output_video_path, output_text_path):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object with reduced frame size
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width // 2, frame_height // 2))

    ball_positions = {}
    events = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # in seconds

        # Convert frame to HSV for better color detection
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Define color ranges for green, orange, white, and yellow
        colors = {
            'green': ((36, 100, 100), (86, 255, 255)),
            'orange': ((5, 50, 50), (15, 255, 255)),
            'white': ((0, 0, 200), (180, 20, 255)),
            'yellow': ((25, 100, 100), (35, 255, 255))
        }

        for color, (lower, upper) in colors.items():
            mask = cv2.inRange(hsv, lower, upper)
            contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            for contour in contours:
                if cv2.contourArea(contour) < 500:
                    continue

                x, y, w, h = cv2.boundingRect(contour)
                center_x, center_y = x + w // 2, y + h // 2

                current_quadrant = get_quadrant(center_x, center_y, frame_width, frame_height)

                if color not in ball_positions:
                    ball_positions[color] = current_quadrant
                    events.append((timestamp, current_quadrant, color, 'Entry'))
                else:
                    previous_quadrant = ball_positions[color]
                    if current_quadrant != previous_quadrant:
                        events.append((timestamp, previous_quadrant, color, 'Exit'))
                        events.append((timestamp, current_quadrant, color, 'Entry'))
                        ball_positions[color] = current_quadrant

                # Draw rectangle around detected ball
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, f'{color} {current_quadrant}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                for event in events:
                    if event[0] == timestamp and event[2] == color:
                        event_text = f'{event[3]} {event[1]} {timestamp:.2f}s'
                        cv2.putText(frame, event_text, (x, y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Resize frame to half size before writing
        resized_frame = cv2.resize(frame, (frame_width // 2, frame_height // 2))
        out.write(resized_frame)

        # Display the frame
        cv2.imshow('Frame', resized_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Save events to a text file
    with open(output_text_path, 'w') as f:
        for event in events:
            f.write(f'{event[0]:.2f}, {event[1]}, {event[2]}, {event[3]}\n')

    # Print collected data
    for event in events:
        print(f'Time: {event[0]:.2f}s, Quadrant: {event[1]}, Color: {event[2]}, Type: {event[3]}')

    # Print details of all detected colors
    color_events = {color: [] for color in colors.keys()}
    for event in events:
        color_events[event[2]].append(event)

    for color, color_event_list in color_events.items():
        print(f'\nDetails for {color} balls:')
        for event in color_event_list:
            print(f'Time: {event[0]:.2f}s, Quadrant: {event[1]}, Type: {event[3]}')

# Path to your video file
video_path = r"C:\Users\Karan Sankhe\Desktop\projectintenrhsip\AI Assignment video.mp4"
output_video_path = r"C:\Users\Karan Sankhe\Desktop\projectintenrhsip\Processed_AI_Assignment_video.avi"
output_text_path = r"C:\Users\Karan Sankhe\Desktop\projectintenrhsip\events.txt"

track_balls(video_path, output_video_path, output_text_path)


Time: 0.00s, Quadrant: 2, Color: white, Type: Entry
Time: 0.03s, Quadrant: 1, Color: orange, Type: Entry
Time: 2.00s, Quadrant: 1, Color: orange, Type: Exit
Time: 2.00s, Quadrant: 4, Color: orange, Type: Entry
Time: 2.00s, Quadrant: 4, Color: yellow, Type: Entry
Time: 2.10s, Quadrant: 4, Color: orange, Type: Exit
Time: 2.10s, Quadrant: 2, Color: orange, Type: Entry
Time: 2.13s, Quadrant: 2, Color: orange, Type: Exit
Time: 2.13s, Quadrant: 4, Color: orange, Type: Entry
Time: 2.20s, Quadrant: 4, Color: orange, Type: Exit
Time: 2.20s, Quadrant: 2, Color: orange, Type: Entry
Time: 2.23s, Quadrant: 2, Color: orange, Type: Exit
Time: 2.23s, Quadrant: 4, Color: orange, Type: Entry
Time: 2.23s, Quadrant: 4, Color: orange, Type: Exit
Time: 2.23s, Quadrant: 2, Color: orange, Type: Entry
Time: 2.26s, Quadrant: 2, Color: orange, Type: Exit
Time: 2.26s, Quadrant: 4, Color: orange, Type: Entry
Time: 2.26s, Quadrant: 4, Color: orange, Type: Exit
Time: 2.26s, Quadrant: 2, Color: orange, Type: Entry
Ti

Time: 15.21s, Quadrant: 1, Color: orange, Type: Entry
Time: 15.21s, Quadrant: 2, Color: white, Type: Exit
Time: 15.21s, Quadrant: 4, Color: white, Type: Entry
Time: 15.21s, Quadrant: 4, Color: white, Type: Exit
Time: 15.21s, Quadrant: 2, Color: white, Type: Entry
Time: 15.24s, Quadrant: 1, Color: orange, Type: Exit
Time: 15.24s, Quadrant: 4, Color: orange, Type: Entry
Time: 15.24s, Quadrant: 4, Color: orange, Type: Exit
Time: 15.24s, Quadrant: 1, Color: orange, Type: Entry
Time: 15.24s, Quadrant: 2, Color: white, Type: Exit
Time: 15.24s, Quadrant: 4, Color: white, Type: Entry
Time: 15.24s, Quadrant: 4, Color: white, Type: Exit
Time: 15.24s, Quadrant: 2, Color: white, Type: Entry
Time: 15.24s, Quadrant: 2, Color: white, Type: Exit
Time: 15.24s, Quadrant: 1, Color: white, Type: Entry
Time: 15.24s, Quadrant: 1, Color: white, Type: Exit
Time: 15.24s, Quadrant: 2, Color: white, Type: Entry
Time: 15.28s, Quadrant: 1, Color: orange, Type: Exit
Time: 15.28s, Quadrant: 4, Color: orange, Type: E

Time: 27.56s, Quadrant: 1, Color: orange, Type: Entry
Time: 27.56s, Quadrant: 1, Color: white, Type: Exit
Time: 27.56s, Quadrant: 4, Color: white, Type: Entry
Time: 27.56s, Quadrant: 4, Color: white, Type: Exit
Time: 27.56s, Quadrant: 2, Color: white, Type: Entry
Time: 27.56s, Quadrant: 2, Color: white, Type: Exit
Time: 27.56s, Quadrant: 1, Color: white, Type: Entry
Time: 27.56s, Quadrant: 1, Color: white, Type: Exit
Time: 27.56s, Quadrant: 2, Color: white, Type: Entry
Time: 27.56s, Quadrant: 2, Color: white, Type: Exit
Time: 27.56s, Quadrant: 1, Color: white, Type: Entry
Time: 27.59s, Quadrant: 1, Color: orange, Type: Exit
Time: 27.59s, Quadrant: 3, Color: orange, Type: Entry
Time: 27.59s, Quadrant: 3, Color: orange, Type: Exit
Time: 27.59s, Quadrant: 4, Color: orange, Type: Entry
Time: 27.59s, Quadrant: 4, Color: orange, Type: Exit
Time: 27.59s, Quadrant: 1, Color: orange, Type: Entry
Time: 27.59s, Quadrant: 1, Color: white, Type: Exit
Time: 27.59s, Quadrant: 4, Color: white, Type: E

Time: 37.18s, Quadrant: 2, Color: white, Type: Exit
Time: 37.18s, Quadrant: 4, Color: white, Type: Entry
Time: 37.18s, Quadrant: 4, Color: white, Type: Exit
Time: 37.18s, Quadrant: 2, Color: white, Type: Entry
Time: 37.18s, Quadrant: 2, Color: white, Type: Exit
Time: 37.18s, Quadrant: 1, Color: white, Type: Entry
Time: 37.18s, Quadrant: 1, Color: white, Type: Exit
Time: 37.18s, Quadrant: 2, Color: white, Type: Entry
Time: 37.21s, Quadrant: 1, Color: orange, Type: Exit
Time: 37.21s, Quadrant: 4, Color: orange, Type: Entry
Time: 37.21s, Quadrant: 4, Color: orange, Type: Exit
Time: 37.21s, Quadrant: 1, Color: orange, Type: Entry
Time: 37.21s, Quadrant: 2, Color: white, Type: Exit
Time: 37.21s, Quadrant: 4, Color: white, Type: Entry
Time: 37.21s, Quadrant: 4, Color: white, Type: Exit
Time: 37.21s, Quadrant: 2, Color: white, Type: Entry
Time: 37.25s, Quadrant: 1, Color: orange, Type: Exit
Time: 37.25s, Quadrant: 4, Color: orange, Type: Entry
Time: 37.25s, Quadrant: 4, Color: orange, Type: E

Time: 50.26s, Quadrant: 4, Color: white, Type: Exit
Time: 50.26s, Quadrant: 2, Color: white, Type: Entry
Time: 50.29s, Quadrant: 2, Color: white, Type: Exit
Time: 50.29s, Quadrant: 4, Color: white, Type: Entry
Time: 50.29s, Quadrant: 4, Color: white, Type: Exit
Time: 50.29s, Quadrant: 2, Color: white, Type: Entry
Time: 50.29s, Quadrant: 2, Color: white, Type: Exit
Time: 50.29s, Quadrant: 4, Color: white, Type: Entry
Time: 50.33s, Quadrant: 4, Color: white, Type: Exit
Time: 50.33s, Quadrant: 2, Color: white, Type: Entry
Time: 50.33s, Quadrant: 2, Color: white, Type: Exit
Time: 50.33s, Quadrant: 4, Color: white, Type: Entry
Time: 50.36s, Quadrant: 4, Color: white, Type: Exit
Time: 50.36s, Quadrant: 2, Color: white, Type: Entry
Time: 50.36s, Quadrant: 2, Color: white, Type: Exit
Time: 50.36s, Quadrant: 4, Color: white, Type: Entry
Time: 50.36s, Quadrant: 4, Color: white, Type: Exit
Time: 50.36s, Quadrant: 2, Color: white, Type: Entry
Time: 50.39s, Quadrant: 2, Color: white, Type: Exit
Tim

...

Time: 28.29s, Quadrant: 2, Type: Exit
Time: 28.29s, Quadrant: 1, Type: Entry
Time: 28.29s, Quadrant: 1, Type: Exit
Time: 28.29s, Quadrant: 2, Type: Entry
Time: 28.33s, Quadrant: 2, Type: Exit
Time: 28.33s, Quadrant: 4, Type: Entry
Time: 28.33s, Quadrant: 4, Type: Exit
Time: 28.33s, Quadrant: 1, Type: Entry
Time: 28.33s, Quadrant: 1, Type: Exit
Time: 28.33s, Quadrant: 2, Type: Entry
Time: 28.36s, Quadrant: 2, Type: Exit
Time: 28.36s, Quadrant: 4, Type: Entry
Time: 28.36s, Quadrant: 4, Type: Exit
Time: 28.36s, Quadrant: 1, Type: Entry
Time: 28.36s, Quadrant: 1, Type: Exit
Time: 28.36s, Quadrant: 2, Type: Entry
Time: 28.39s, Quadrant: 2, Type: Exit
Time: 28.39s, Quadrant: 4, Type: Entry
Time: 28.39s, Quadrant: 4, Type: Exit
Time: 28.39s, Quadrant: 1, Type: Entry
Time: 28.39s, Quadrant: 1, Type: Exit
Time: 28.39s, Quadrant: 2, Type: Entry
Time: 28.43s, Quadrant: 2, Type: Exit
Time: 28.43s, Quadrant: 4, Type: Entry
Time: 28.43s, Quadrant: 4, Type: Exit
Time: 28.43s, Quadrant: 2, Type: Entry

Time: 46.67s, Quadrant: 2, Type: Entry
Time: 46.67s, Quadrant: 2, Type: Exit
Time: 46.67s, Quadrant: 1, Type: Entry
Time: 46.67s, Quadrant: 1, Type: Exit
Time: 46.67s, Quadrant: 2, Type: Entry
Time: 46.67s, Quadrant: 2, Type: Exit
Time: 46.67s, Quadrant: 1, Type: Entry
Time: 46.70s, Quadrant: 1, Type: Exit
Time: 46.70s, Quadrant: 4, Type: Entry
Time: 46.70s, Quadrant: 4, Type: Exit
Time: 46.70s, Quadrant: 2, Type: Entry
Time: 46.70s, Quadrant: 2, Type: Exit
Time: 46.70s, Quadrant: 1, Type: Entry
Time: 46.73s, Quadrant: 1, Type: Exit
Time: 46.73s, Quadrant: 4, Type: Entry
Time: 46.73s, Quadrant: 4, Type: Exit
Time: 46.73s, Quadrant: 2, Type: Entry
Time: 46.73s, Quadrant: 2, Type: Exit
Time: 46.73s, Quadrant: 1, Type: Entry
Time: 46.77s, Quadrant: 1, Type: Exit
Time: 46.77s, Quadrant: 4, Type: Entry
Time: 46.77s, Quadrant: 4, Type: Exit
Time: 46.77s, Quadrant: 2, Type: Entry
Time: 46.77s, Quadrant: 2, Type: Exit
Time: 46.77s, Quadrant: 1, Type: Entry
Time: 46.80s, Quadrant: 1, Type: Exit

Time: 62.71s, Quadrant: 4, Type: Entry
Time: 62.71s, Quadrant: 4, Type: Exit
Time: 62.71s, Quadrant: 2, Type: Entry
Time: 62.74s, Quadrant: 2, Type: Exit
Time: 62.74s, Quadrant: 4, Type: Entry
Time: 62.74s, Quadrant: 4, Type: Exit
Time: 62.74s, Quadrant: 3, Type: Entry
Time: 62.74s, Quadrant: 3, Type: Exit
Time: 62.74s, Quadrant: 4, Type: Entry
Time: 62.74s, Quadrant: 4, Type: Exit
Time: 62.74s, Quadrant: 2, Type: Entry
Time: 62.74s, Quadrant: 2, Type: Exit
Time: 62.74s, Quadrant: 4, Type: Entry
Time: 62.74s, Quadrant: 4, Type: Exit
Time: 62.74s, Quadrant: 2, Type: Entry
Time: 62.78s, Quadrant: 2, Type: Exit
Time: 62.78s, Quadrant: 4, Type: Entry
Time: 62.78s, Quadrant: 4, Type: Exit
Time: 62.78s, Quadrant: 3, Type: Entry
Time: 62.78s, Quadrant: 3, Type: Exit
Time: 62.78s, Quadrant: 4, Type: Entry
Time: 62.78s, Quadrant: 4, Type: Exit
Time: 62.78s, Quadrant: 2, Type: Entry
Time: 62.81s, Quadrant: 2, Type: Exit
Time: 62.81s, Quadrant: 4, Type: Entry
Time: 62.81s, Quadrant: 4, Type: Exit

Time: 77.02s, Quadrant: 2, Type: Exit
Time: 77.02s, Quadrant: 4, Type: Entry
Time: 77.02s, Quadrant: 4, Type: Exit
Time: 77.02s, Quadrant: 2, Type: Entry
Time: 77.02s, Quadrant: 2, Type: Exit
Time: 77.02s, Quadrant: 4, Type: Entry
Time: 77.02s, Quadrant: 4, Type: Exit
Time: 77.02s, Quadrant: 2, Type: Entry
Time: 77.06s, Quadrant: 2, Type: Exit
Time: 77.06s, Quadrant: 4, Type: Entry
Time: 77.06s, Quadrant: 4, Type: Exit
Time: 77.06s, Quadrant: 2, Type: Entry
Time: 77.06s, Quadrant: 2, Type: Exit
Time: 77.06s, Quadrant: 4, Type: Entry
Time: 77.06s, Quadrant: 4, Type: Exit
Time: 77.06s, Quadrant: 2, Type: Entry
Time: 77.09s, Quadrant: 2, Type: Exit
Time: 77.09s, Quadrant: 4, Type: Entry
Time: 77.09s, Quadrant: 4, Type: Exit
Time: 77.09s, Quadrant: 2, Type: Entry
Time: 77.09s, Quadrant: 2, Type: Exit
Time: 77.09s, Quadrant: 4, Type: Entry
Time: 77.09s, Quadrant: 4, Type: Exit
Time: 77.09s, Quadrant: 2, Type: Entry
Time: 77.12s, Quadrant: 2, Type: Exit
Time: 77.12s, Quadrant: 4, Type: Entry

Time: 90.17s, Quadrant: 4, Type: Exit
Time: 90.17s, Quadrant: 2, Type: Entry
Time: 90.17s, Quadrant: 2, Type: Exit
Time: 90.17s, Quadrant: 4, Type: Entry
Time: 90.20s, Quadrant: 4, Type: Exit
Time: 90.20s, Quadrant: 2, Type: Entry
Time: 90.20s, Quadrant: 2, Type: Exit
Time: 90.20s, Quadrant: 4, Type: Entry
Time: 90.20s, Quadrant: 4, Type: Exit
Time: 90.20s, Quadrant: 2, Type: Entry
Time: 90.24s, Quadrant: 2, Type: Exit
Time: 90.24s, Quadrant: 4, Type: Entry
Time: 90.24s, Quadrant: 4, Type: Exit
Time: 90.24s, Quadrant: 2, Type: Entry
Time: 90.24s, Quadrant: 2, Type: Exit
Time: 90.24s, Quadrant: 4, Type: Entry
Time: 90.24s, Quadrant: 4, Type: Exit
Time: 90.24s, Quadrant: 2, Type: Entry
Time: 90.27s, Quadrant: 2, Type: Exit
Time: 90.27s, Quadrant: 4, Type: Entry
Time: 90.27s, Quadrant: 4, Type: Exit
Time: 90.27s, Quadrant: 2, Type: Entry
Time: 90.30s, Quadrant: 2, Type: Exit
Time: 90.30s, Quadrant: 4, Type: Entry
Time: 90.30s, Quadrant: 4, Type: Exit
Time: 90.30s, Quadrant: 2, Type: Entry